<a href="https://colab.research.google.com/github/amandatz/linear-programming/blob/main/Atividade3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 3



Amanda Topanotti Zanette (22100776)

**Importações e funções auxiliares**

In [1]:
using Pkg
Pkg.activate("pastesian")
Pkg.add("JuMP")
Pkg.add("HiGHS")

  Activating new project at `/content/pastesian`
   Resolving package versions...
   Installed ForwardDiff ─ v1.3.0
    Updating `/content/pastesian/Project.toml`
  [4076af6c] + JuMP v1.29.3
    Updating `/content/pastesian/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.3
  [523fee87] + CodecBzip2 v0.8.5
  [944b1d66] + CodecZlib v0.7.8
  [bbf7d656] + CommonSubexpressions v0.3.1
  [34da2185] + Compat v4.18.1
  [864edb3b] + DataStructures v0.19.3
  [163ba53b] + DiffResults v1.1.0
  [b552c78f] + DiffRules v1.15.1
  [ffbed154] + DocStringExtensions v0.9.5
  [f6369f11] + ForwardDiff v1.3.0
  [92d709cd] + IrrationalConstants v0.2.6
  [692b3bcd] + JLLWrappers v1.7.1
  [682c06a0] + JSON v1.3.0
  [0f8b85d8] + JSON3 v1.14.3
  [4076af6c] + JuMP v1.29.3
  [2ab3a3ac] + LogExpFunctions v0.3.29
  [1914dd2f] + MacroTools v0.5.16
  [b8f27783] + MathOptInterface v1.46.0
  [d8a4904e] + MutableArithmetics v1.6.7
  [77ba4419] + NaNMath v1.1.3
  [bac558e1] + OrderedCollections v1.8.1
  [69de0a69] + Parse

In [2]:
using JuMP, HiGHS, LinearAlgebra, Printf

## Problema

A Pastesian é uma fábrica de massas familiar que está planejando a produção de lasanhas para os próximos 4 meses. Além do sabor tradicional, ela decidiu lançar um novo sabor de lasanha para esta temporada. A empresa planeja suas operações ao longo de 4 meses. Para o sabor tradicional, a demanda para os primeiros 3 meses é considerada conhecida com base em dados históricos: 200, 350 e 150 unidades, respectivamente.

A demanda do 4° mês para o sabor tradicional é mais incerta, pois coincide com a alta temporada na região e historicamente apresenta grande variabilidade. Para lidar com essa incerteza, Ricardo, o dono da Pastesian, trabalha com três cenários possíveis para a demanda do sabor tradicional no último mês, dados por:

- Cenário 1: 220 unidades  
- Cenário 2: 250 unidades  
- Cenário 3: 300 unidades  

Assume-se que esses cenários são equiprováveis, e o modelo de produção deve ser capaz de acomodar essas variações sem incorrer em custos excessivos de produção, estocagem ou falta de produto.

Paralelamente, o novo sabor de lasanha também precisa ser planejado. Estima-se que a demanda desse novo produto seja de 30, 70 e 140 unidades nos meses 1, 2 e 3, respectivamente. Para o 4° mês, a demanda pode assumir os seguintes valores, com as probabilidades esperadas indicadas a seguir:

| Demanda (unidades) | Probabilidade |
|--------------------|---------------|
| 200                | 0,3           |
| 240                | 0,5           |
| 300                | 0,2           |

Além disso, Ricardo pretende expandir gradualmente a infraestrutura de armazenamento ao longo da temporada. Dessa forma, a capacidade total de estoque da fábrica (somando lasanha tradicional e nova) em cada mês é limitada por:

| Mês | Capacidade de estoque (unidades) |
|-----|----------------------------------|
| 1   | 200                              |
| 2   | 220                              |
| 3   | 230                              |
| 4   | 250                              |

O objetivo é determinar, para cada mês e para cada sabor, quanto produzir e quanto manter em estoque, de forma a minimizar o custo total de operação (produção, estocagem e eventuais faltas), respeitando as limitações impostas.

## Parâmetros

In [3]:
# Períodos (mês)
T = 1:4;
# Custo (R$/unidade) de produção em cada mês
cp = [5.5, 7.2, 8.8, 10.9];
# Custo (R$/unidade) de manter as lasanhas em estoque, de um mês para o seguinte
ce = [1.3, 1.95, 2.2];
# Capacidade total de estoque em cada mês (para as duas lasanhas somadas)
E = [200, 220, 230, 250];

# Modelo para lasanha original

**Parâmetros**

In [4]:
# Demanda certa nos meses 1–3 (lasanha original)
D_o = [200, 350, 150];
# Lasanhas originais (unidade) em estoque inicialmente
e0_o = 50;

# Cenários para o mês 4
C_o = 1:3;
# Demanda incerta no mês 4
D4_o = [220, 250, 300]
# Probabilidade de cada cenário Original = 1/3
Po = 1.0 / length(D4_o)
# Penalidade por demanda não atendida no mês 4
pp_o = 5.045 # ONDE ENTRA ISSO????

5.045

**Modelo**

Esta função constrói a estrutura da Lasanha Original, contendo apenas os elementos fixos e comuns aos problemas subsequentes. Ela representa a base necessária para o planejamento dos meses com demanda certa (Mês 1, 2 e 3) e será *eutilizada tanto na formulação do modelo determinístico quanto no modelo estocástico.

Observe que a função objetivo e o tratamento da incerteza do mês 4 não são definidos aqui.

O foco desta função base está apenas nas restrições estruturais da lasanha original para os primeiros três meses:

* **Estoque inicial:** Fixa o estoque da lasanha original no início da temporada: `e_o[1] = e0_o`;
* **Balanço de estoque:** Define a relação de fluxo entre produção, demanda e estoque ao longo dos três primeiros meses: `e_o[t+1] = e_o[t] + l_o[t] - D_orig[t]` para $t \in \{1, 2, 3\}$.

As variáveis de produção ($\mathbf{l}_o[4]$) e estoque ($\mathbf{e}_o[4]$) do Mês 4 são criadas neste ponto, mas suas restrições serão adicionadas na próxima etapa de cada modelo.

In [5]:
function build_base_original(model, D_o)

    # === VARIÁVEIS ===
    @variable(model, l_o[1:4] >= 0)      # Produção da lasanha original
    @variable(model, e_o[0:4] >= 0)      # Estoque da lasanha original

    # === RESTRIÇÕES (Meses 1, 2, 3) ===

    # Estoque inicial
    @constraint(model, e_o[0] == e0_o)

    # Balanço meses 1-3: estoque_final[t] = estoque_final[t-1] + produção[t] - demanda[t]
    @constraint(model, BAL_o[t in 1:3],
        e_o[t] == e_o[t-1] + l_o[t] - D_o[t]
    )

    return model, l_o, e_o
end

build_base_original (generic function with 1 method)

# Modelo para lasanha nova

**Parâmetros**

In [6]:
# Demanda certa nos meses 1–3 (lasanha nova)
D_n = [30, 70, 140];

# Cenários para o mês 4
C_n = 1:3;
# Demanda incerta no mês 4
D4_n = [200, 240, 300];
# Probabilidades
P4_n = [0.3, 0.5, 0.2];
# Estoque inicial
e0_n = 0;

In [7]:
function build_base_nova(model, D_n)
    # === VARIÁVEIS ===
    @variable(model, l_n[1:4] >= 0)      # Produção
    @variable(model, e_n[0:4] >= 0)      # Estoque (0=inicial, 1-4=final de cada mês)

    # === RESTRIÇÕES (Meses 1-3) ===

    # Estoque inicial zero
    @constraint(model, e_n[0] == e0_n)

    # Balanço meses 1-3
    @constraint(model, BAL_n[t in 1:3],
        e_n[t] == e_n[t-1] + l_n[t] - D_n[t]
    )

    return model, l_n, e_n
end

build_base_nova (generic function with 1 method)

## Modelo determinístico

O modelo determinístico para o novo sabor de lasanha é construído utilizando a demanda média esperada para o mês 4. A demanda incerta do mês 4 é resolvida pelo seu valor médio esperado, `d4_mean_n = 240` unidades.

In [8]:
d4_mean_n = dot(D4_n, P4_n);
D_n_det = vcat(D_n, d4_mean_n);

**Modelo**

O modelo é iniciado e as variáveis do sabor original são carregadas.

In [18]:
model_det = Model(HiGHS.Optimizer);
model_det, l_o, e_o = build_base_original(model_det, D_o);
model_det, l_n, e_n = build_base_nova(model_det, D_n);

As seguintes variáveis são introduzidas para o novo sabor:

In [19]:
# === VARIÁVEIS ADICIONAIS - MÊS 4 ===
# Déficit por cenário (lasanha original)
@variable(model_det, dp_o[C_o] >= 0)


1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{VariableRef}:
 dp_o[1]
 dp_o[2]
 dp_o[3]

In [20]:
# === RESTRIÇÕES - MÊS 4 ===

# Balanço lasanha nova (determinístico com demanda média)
@constraint(model_det, BAL_n_4,
    e_n[4] == e_n[3] + l_n[4] - d4_mean_n
)

# Atendimento lasanha original por cenário (permite déficit)
# Produção[4] + Estoque[3] >= Demanda[cenário] - Déficit[cenário]
@constraint(model_det, DEM_o[c in C_o],
    l_o[4] + e_o[3] >= D4_o[c] - dp_o[c]
)

# Balanço lasanha original (usa demanda média para calcular estoque final)
d4_mean_o = sum(Po * D4_o[c] for c in C_o)
@constraint(model_det, BAL_o_4,
    e_o[4] == e_o[3] + l_o[4] - d4_mean_o
)

BAL_o_4 : -l_o[4] - e_o[3] + e_o[4] = -256.66666666666663

Além disso, o armazenamento é compartilhada entre ambos os tipos de lasanhas. Logo precisamos introduzir a seguinte restrição:

In [21]:
# === RESTRIÇÕES - CAPACIDADE DE ESTOQUE ===
# Capacidade no final de cada mês (estoque somado)
@constraint(model_det, CE[t in 1:4],
    e_o[t] + e_n[t] <= E[t]
)


4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 CE[1] : e_o[1] + e_n[1] ≤ 200
 CE[2] : e_o[2] + e_n[2] ≤ 220
 CE[3] : e_o[3] + e_n[3] ≤ 230
 CE[4] : e_o[4] + e_n[4] ≤ 250

A função objetivo é a soma dos custos determinísticos do novo sabor com o custo esperado dos componentes estocásticos do sabor original.

In [22]:
custo_1_3 = (
    sum(cp[t] * (l_o[t] + l_n[t]) for t in 1:3)  # Produção
    + sum(ce[t] * (e_o[t] + e_n[t]) for t in 1:3)  # Estoque
)

# Custo esperado mês 4
custo_mes_4 = (
    cp[4] * l_o[4]  # Produção original
    + cp[4] * l_n[4]  # Produção nova
    + sum(Po * pp_o * dp_o[c] for c in C_o)  # Penalidade esperada
)

@objective(model_det, Min, custo_1_3 + custo_mes_4)

5.5 l_o[1] + 5.5 l_n[1] + 7.2 l_o[2] + 7.2 l_n[2] + 8.8 l_o[3] + 8.8 l_n[3] + 1.3 e_o[1] + 1.3 e_n[1] + 1.95 e_o[2] + 1.95 e_n[2] + 2.2 e_o[3] + 2.2 e_n[3] + 10.9 l_o[4] + 10.9 l_n[4] + 1.6816666666666666 dp_o[1] + 1.6816666666666666 dp_o[2] + 1.6816666666666666 dp_o[3]

In [23]:
optimize!(model_det)

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 17 rows; 21 cols; 43 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [1e+00, 1e+01]
  Bound   [0e+00, 0e+00]
  RHS     [3e+01, 4e+02]
Presolving model
13 rows, 17 cols, 35 nonzeros  0s
Dependent equations search running on 6 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
13 rows, 17 cols, 35 nonzeros  0s
Presolve reductions: rows 13(-4); columns 17(-4); nonzeros 35(-8) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     9.9000000000e+02 Pr: 9(1976.67) 0s
         13     1.1972538889e+04 Pr: 0(0); Du: 0(8.88178e-16) 0s

Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Simplex   iterations: 13
Objective value     :  1.1972538889e+04
P-D objective error :

In [24]:
println("Status: ", termination_status(model_det))
println("FO ótima = ", objective_value(model_det))

println("\nProdução lasanha original (l_o): ", value.(l_o))
println("Estoque lasanha original (e_o):   ", value.(e_o))

println("\nProdução lasanha nova (l_n):      ", value.(l_n))
println("Estoque lasanha nova (e_n):        ", value.(e_n))

Status: OPTIMAL
FO ótima = 11972.538888888888

Produção lasanha original (l_o): [350.0, 150.0, 150.0, 256.66666666666663]
Estoque lasanha original (e_o):   1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:4
And data, a 5-element Vector{Float64}:
  50.0
 200.0
   0.0
   0.0
   0.0

Produção lasanha nova (l_n):      [30.0, 70.0, 140.0, 240.0]
Estoque lasanha nova (e_n):        1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:4
And data, a 5-element Vector{Float64}:
 -0.0
  0.0
  0.0
  0.0
  0.0


### 1.5. Análise de sensibilidade das restrições de capacidade de estoque


In [16]:
report = lp_sensitivity_report(model_det)

SensitivityReport{Float64}(Dict{ConstraintRef, Tuple{Float64, Float64}}(dp_o[3] ≥ 0 => (-Inf, Inf), dp_o[2] ≥ 0 => (-Inf, Inf), dp_o[1] ≥ 0 => (-Inf, Inf)), Dict{VariableRef, Tuple{Float64, Float64}}(dp_o[2] => (-Inf, 0.0), dp_o[1] => (-Inf, 0.0), dp_o[3] => (-Inf, 0.0)))

# Modelo estocástico para lasanha nova

Como a incerteza no Mês 4 é dada por cenários de demanda independentes para a lasanha original ($\mathbf{C}_o = 3$ cenários) e a lasanha nova ($\mathbf{C}_n = 3$ cenários), o conjunto total de cenários estocásticos é:
$$
  S = \mathbf{C}_o \times \mathbf{C}_n \implies |S| = 3 \times 3 = 9 \text{ cenários.}
$$

Cada cenário $s \in S$ tem uma probabilidade $\mathbf{P}[s] = \mathbf{P}[s_o] \cdot \mathbf{P}[s_n]$.

**Parâmetros**

**Modelo**

In [17]:
model_estocastico = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none